In [14]:
import pandas as pd
import numpy as np

# LOAD DATA
ratings = pd.read_csv("/Users/aryamane/Downloads/final_ratings_cleaned.csv", parse_dates=['timestamp'])
users = pd.read_csv("/Users/aryamane/Downloads/UserData/User_data_final.csv", parse_dates=['signup_date'])

ratings = ratings.merge(users[['user_id', 'signup_date']], on='user_id', how='left')

# SORTING FOR TIME GAP CALCULATION 
ratings = ratings.sort_values(by=['user_id', 'timestamp'])
ratings['prev_time'] = ratings.groupby('user_id')['timestamp'].shift()
ratings['gap_days'] = (ratings['timestamp'] - ratings['prev_time']).dt.days

# USER-LEVEL FEATURE AGGREGATION 
user_features = ratings.groupby('user_id').agg({
    'rating': ['count', 'mean', 'std'],
    'timestamp': ['min', 'max'],
    'gap_days': 'mean'
})

# FLATTEN MULTI-LEVEL COLUMNS
user_features.columns = [
    'num_ratings', 'avg_rating', 'std_rating',
    'first_rating_date', 'last_rating_date', 'avg_days_between_ratings'
]
user_features = user_features.reset_index()

user_features['days_active'] = (
    user_features['last_rating_date'] - user_features['first_rating_date']
).dt.days

users['signup_date'] = pd.to_datetime(users['signup_date'])
user_signup = users[['user_id', 'signup_date']].copy()
user_signup['signup_year'] = user_signup['signup_date'].dt.year

user_features = user_features.merge(user_signup[['user_id', 'signup_year']], on='user_id', how='left')

user_features = user_features.drop(columns=['first_rating_date', 'last_rating_date'])
user_features['std_rating'] = user_features['std_rating'].fillna(0)
user_features['avg_days_between_ratings'] = user_features['avg_days_between_ratings'].fillna(0)

user_features.to_csv("/Users/aryamane/Downloads/user_features.csv", index=False)
print("user_features.csv created successfully!")


/var/folders/k2/5hcbwx856_z5_q1m7ly2q4mm0000gn/T/ipykernel_25135/1611621295.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ratings = pd.read_csv("/Users/aryamane/Downloads/final_ratings_cleaned.csv", parse_dates=['timestamp'])


user_features.csv created successfully!


/var/folders/k2/5hcbwx856_z5_q1m7ly2q4mm0000gn/T/ipykernel_25135/1611621295.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  users = pd.read_csv("/Users/aryamane/Downloads/UserData/User_data_final.csv", parse_dates=['signup_date'])
